In [ ]:
!nvidia-smi

In [ ]:
!pip uninstall -y numpy scipy scikit-learn albumentations
!pip install numpy==1.26.4 scipy==1.11.4 matplotlib==3.9.2 scikit-learn==1.3.2 albumentations==1.4.3 --force-reinstall --no-cache-dir

In [ ]:
!pip install ultralytics opencv-python tqdm pyyaml torch torchvision

In [ ]:
!pip install --upgrade --force-reinstall pillow ultralytics

In [ ]:
import json
import os
from tqdm import tqdm
import shutil
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import yaml
from pathlib import Path
import torch
from ultralytics import YOLO

In [ ]:
source_path = "/kaggle/input/flir-thermal-images-dataset/FLIR_ADAS_1_3/train/thermal_annotations.json"
destination_path = "/kaggle/working/flir_yolo/train/labels"

source_path2 = "/kaggle/input/flir-thermal-images-dataset/FLIR_ADAS_1_3/val/thermal_annotations.json"
destination_path2 = "/kaggle/working/flir_yolo/val/labels"

img_width = 640
img_height = 512

def convert(source_path, destination_path, img_width=640, img_height=512):
    os.makedirs(destination_path, exist_ok=True)

    with open(source_path, 'r') as f:
        data = json.load(f)

    PERSON_FLIR_ID = 1
    YOLO_CLASS_ID = 0
    
    print(f"Loaded {len(data['annotations'])} annotations from {source_path}")
    print(f"Converting to SINGLE YOLO CLASS (Person={YOLO_CLASS_ID}) format...\n")
    
    image_map = {img["id"]: img["file_name"] for img in data["images"]}

    counts = 0
    images_processed = set()

    for ann in tqdm(data["annotations"], desc="Processing Annotations"):
        cat_id = ann["category_id"]
        
        if cat_id != PERSON_FLIR_ID:
            continue

        img_id = ann["image_id"]
        if img_id not in image_map:
            continue

        img_name = os.path.splitext(os.path.basename(image_map[img_id]))[0]
        label_file = os.path.join(destination_path, f"{img_name}.txt")

        x, y, w, h = ann["bbox"]
        
        if w <= 0 or h <= 0:
             continue
        
        if x < 0 or y < 0 or x + w > img_width or y + h > img_height:
             continue
        x_center = (x + w / 2) / img_width
        y_center = (y + h / 2) / img_height
        w /= img_width
        h /= img_height
        
        if not (0.0 <= x_center <= 1.0 and 0.0 <= y_center <= 1.0 and 0.0 <= w <= 1.0 and 0.0 <= h <= 1.0):
             continue 

        with open(label_file, "a") as f:
            f.write(f"{YOLO_CLASS_ID} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

        counts += 1
        images_processed.add(img_name)

    print(f"\nConversion complete! {counts} annotations written for {len(images_processed)} images in {destination_path}")


convert(source_path, destination_path, img_width, img_height)
convert(source_path2, destination_path2, img_width, img_height)
        

In [ ]:
import os

yaml_path = "/kaggle/working/flir_yolo/data.yaml"

os.makedirs("/kaggle/working/flir_yolo", exist_ok=True)

with open(yaml_path, "w") as f:
    f.write("""path: /kaggle/working/flir_yolo

train: train/images
val: val/images

names:
  0: person
""")

print(f"data.yaml created at: {yaml_path}")

In [ ]:
train_img_src = "/kaggle/input/flir-thermal-images-dataset/FLIR_ADAS_1_3/train/thermal_8_bit"
val_img_src = "/kaggle/input/flir-thermal-images-dataset/FLIR_ADAS_1_3/val/thermal_8_bit"

train_img_dst = "/kaggle/working/flir_yolo/train/images"
val_img_dst = "/kaggle/working/flir_yolo/val/images"

os.makedirs(train_img_dst, exist_ok=True)
os.makedirs(val_img_dst, exist_ok=True)

for src, dst in [(train_img_src, train_img_dst), (val_img_src, val_img_dst)]:
    for fname in os.listdir(src):
        os.symlink(os.path.join(src, fname), os.path.join(dst, fname))


print("Image folders ready!")

In [ ]:
class Config:
    ORIGINAL_DATASET_PATH = "/kaggle/working/flir_yolo"
    VAL_PATH = "/kaggle/working/flir_yolo/val"
    OUTPUT_PATH = "/kaggle/working/flair_fire_augmented"
    PRETRAINED_MODEL = "/kaggle/input/test-d/T3/train/weights/best.pt"

    
    NUM_IMAGES_train = 8000
    NUM_IMAGES_val = 1000
    
    IMG_SIZE = 640
    BATCH_SIZE = 16
    EPOCHS = 20
        
    FIRE_INTENSITY = 0.7

# ==================== FIRE ENVIRONMENT AUGMENTATION ====================
class FireEnvironmentAugmentor:    
    def __init__(self, intensity=0.7):
        self.intensity = intensity
        
    def add_smoke_effect(self, image):
        h, w = image.shape[:2]
        
        smoke = np.random.rand(h, w).astype(np.float32)
        smoke = cv2.GaussianBlur(smoke, (51, 51), 0)
        smoke = (smoke * 255 * self.intensity * 0.6).astype(np.uint8)
        
        smoke_3ch = cv2.cvtColor(smoke, cv2.COLOR_GRAY2BGR)
        result = cv2.addWeighted(image, 1.0, smoke_3ch, 0.4, 0)
        
        return result

    def add_fire_glow(self, image):
        h, w = image.shape[:2]
        num_glows = np.random.randint(3, 6)
        fire_overlay = np.zeros((h, w, 3), dtype=np.float32)
    
        for _ in range(num_glows):
            cx, cy = np.random.randint(0, w), np.random.randint(0, h)
            radius = np.random.randint(30, 100)
            color = np.random.uniform(0.8, 1.0, 3) * np.array([1.0, 0.6, 0.2])  # orange-ish
            cv2.circle(fire_overlay, (cx, cy), radius, color.tolist(), -1)
    
        fire_overlay = cv2.GaussianBlur(fire_overlay, (0, 0), sigmaX=20, sigmaY=20)
    
        y, x = np.ogrid[:h, :w]
        center_y, center_x = h // 2, w // 2
        distance = np.sqrt((x - center_x)**2 + (y - center_y)**2)
        intensity_map = 1 - np.clip(distance / max(h, w), 0, 1)
        intensity_map = intensity_map[..., None]  # add channel dim
    
        fire_overlay *= intensity_map * self.intensity * 0.3
        fire_overlay = fire_overlay.astype(np.float32)
        image = image.astype(np.float32)
    
        result = cv2.addWeighted(image, 1.0, fire_overlay, 1.0, 0.0)
        return np.clip(result, 0, 255).astype(np.uint8)

    
    def add_heat_shimmer(self, image):
        kernel_size = int(5 + 10 * self.intensity)
        if kernel_size % 2 == 0:
            kernel_size += 1
        
        angle = np.random.randint(0, 180)
        kernel = np.zeros((kernel_size, kernel_size))
        kernel[kernel_size // 2, :] = 1
        
        center = (kernel_size // 2, kernel_size // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        kernel = cv2.warpAffine(kernel, rotation_matrix, (kernel_size, kernel_size))
        kernel = kernel / kernel.sum()
        
        result = cv2.filter2D(image, -1, kernel)
        return result
    
    def reduce_visibility(self, image):
        alpha = 1.0 - (0.3 * self.intensity)  # Contrast
        beta = 20 * self.intensity  # Brightness increase
        
        result = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
        return result
    
    def apply_fire_augmentation(self, image):
        image = self.add_smoke_effect(image)
        image = self.add_fire_glow(image)
        image = self.add_heat_shimmer(image)
        image = self.reduce_visibility(image)
        
        return image

# ==================== ALBUMENTATIONS PIPELINE ====================
def get_fire_training_transforms(img_size=640):
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.3),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, 
                           rotate_limit=15, p=0.5),
        
        A.RandomBrightnessContrast(brightness_limit=0.4, 
                                   contrast_limit=0.4, p=0.8),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=40, 
                            val_shift_limit=40, p=0.7),
        
        A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.5, 
                   alpha_coef=0.1, p=0.4),
        A.GaussianBlur(blur_limit=(3, 7), p=0.3),
        A.MotionBlur(blur_limit=7, p=0.3),
        
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
        A.ISONoise(p=0.2),
        
        A.RandomGamma(gamma_limit=(80, 120), p=0.3),
        A.CLAHE(clip_limit=4.0, p=0.3),
        
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, 
                      border_mode=cv2.BORDER_CONSTANT, value=0),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# ==================== DATASET PREPARATION ====================
def prepare_fire_dataset(config):        
    output_path = Path(config.OUTPUT_PATH)
    (output_path / 'images' / 'train').mkdir(parents=True, exist_ok=True)
    (output_path / 'labels' / 'train').mkdir(parents=True, exist_ok=True)
    (output_path / 'images' / 'val').mkdir(parents=True, exist_ok=True)
    (output_path / 'labels' / 'val').mkdir(parents=True, exist_ok=True)
    
    fire_aug = FireEnvironmentAugmentor(intensity=config.FIRE_INTENSITY)
    
    # --------------- TRAIN IMAGES ----------------
    original_train_images = Path(config.ORIGINAL_DATASET_PATH) / 'train' / 'images'
    original_train_labels = Path(config.ORIGINAL_DATASET_PATH) / 'train' / 'labels'
    
    train_images = sorted(list(original_train_images.glob('*.*')))[:config.NUM_IMAGES_train]
    
    print(f"Processing {len(train_images)} train images...")
    for img_path in tqdm(train_images, desc="Augmenting train images"):
        image = cv2.imread(str(img_path))
        if image is None:
            continue
        fire_image = fire_aug.apply_fire_augmentation(image)
        
        output_img_path = output_path / 'images' / 'train' / img_path.name
        cv2.imwrite(str(output_img_path), fire_image)
        
        label_path = original_train_labels / img_path.with_suffix('.txt').name
        if label_path.exists():
            shutil.copy(label_path, output_path / 'labels' / 'train' / label_path.name)
    
    # --------------- VALIDATION IMAGES ----------------
    original_val_images = Path(config.VAL_PATH) / 'images'
    original_val_labels = Path(config.VAL_PATH) / 'labels'
    
    val_images = sorted(list(original_val_images.glob('*.*')))[:config.NUM_IMAGES_val]
    
    print(f"Processing {len(val_images)} validation images...")
    for img_path in tqdm(val_images, desc="Copying val images"):
        shutil.copy(img_path, output_path / 'images' / 'val' / img_path.name)
        
        label_path = original_val_labels / img_path.with_suffix('.txt').name
        if label_path.exists():
            shutil.copy(label_path, output_path / 'labels' / 'val' / label_path.name)
    
    print(f"Dataset prepared at: {output_path}")
    return output_path


# ==================== CREATE YAML CONFIG ====================
def create_dataset_yaml(dataset_path, config):
    """Create YAML configuration for YOLO training"""
    
    yaml_content = {
        'train': 'images/train',
        'val': 'images/val',
        'nc': 1,
        'names': ['person']
    }
    
    yaml_path = dataset_path / 'data.yaml'
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)
    
    print(f"Created dataset config: {yaml_path}")
    return yaml_path


# ==================== TRAINING ====================
def train_fire_yolo(config, dataset_yaml):    
    print("\nStarting YOLO Training on Fire-Augmented Dataset...")
    
    model = YOLO(config.PRETRAINED_MODEL)
    
    results = model.train(
        data=str(dataset_yaml),
        epochs=config.EPOCHS,
        imgsz=config.IMG_SIZE,
        batch=config.BATCH_SIZE,
        
        patience=10,
        save=True,
        project='fire_yolo_training',
        name='fire_detection',
        
        optimizer='AdamW',
        lr0=0.001,
        lrf=0.01,
        
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=10.0,
        translate=0.1,
        scale=0.5,
        shear=0.0,
        perspective=0.0,
        flipud=0.0,
        fliplr=0.5,
        mosaic=1.0,
        mixup=0.1,
        
        device=0 if torch.cuda.is_available() else 'cpu',
        workers=8,
        
        val=True,
        plots=True
    )
    return model

# ==================== MAIN EXECUTION ====================
def main():
    config = Config()
    
    print("=" * 60)
    print("🔥 FIRE ENVIRONMENT YOLO RETRAINING PIPELINE")
    print("=" * 60)
    
    dataset_path = prepare_fire_dataset(config)
    
    yaml_path = create_dataset_yaml(dataset_path, config)
    
    model = train_fire_yolo(config, yaml_path)
    
    best_model_path = Path('fire_yolo_training/fire_detection/weights/best.pt')
    if best_model_path.exists():
        print(f"\n🎯 Best model saved at: {best_model_path}")
    
    print("\n" + "=" * 60)
    print("Pipeline completed successfully!")
    print("=" * 60)

if __name__ == "__main__":
    main()